In [2]:
import pandas as pd
import twitter
import internetarchive as ia
import requests
import json
import os
import datetime as dt
from git import Repo
import locale
locale.setlocale(locale.LC_TIME, 'es_US.UTF8');

In [3]:
datos_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRZv851TVHTZd99eke7VVb3tchFjrp1pwmmK0ipQruVoAHovoDe8_VMgQtDZIPckn6_Aiu5Hux_ACzq/pub?gid=0&single=true&output=csv"
metadatos_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRZv851TVHTZd99eke7VVb3tchFjrp1pwmmK0ipQruVoAHovoDe8_VMgQtDZIPckn6_Aiu5Hux_ACzq/pub?gid=1026774595&single=true&output=csv'
directory = 'reportes'

In [1]:
def archivar_reportes(actualizar=True):
    "Sube al internetarchive las imágenes de reportes citadas en la tabla de metadatos. Si actualizar es `True`, sólo sube aquellas que no hayan sido subidas previamente."
    
    def load_credenciales():
        with open('credenciales.json', 'r') as f:
            return json.load(f)
    
    def auth_twitter():
        "Autenticar en Twitter para realizar consultas al API"

        return twitter.Api(consumer_key = credenciales['twitter']['consumer_key'],
                           consumer_secret = credenciales['twitter']['consumer_secret'],
                           access_token_key = credenciales['twitter']['access_token_key'],
                           access_token_secret = credenciales['twitter']['access_token_secret'],
                           tweet_mode='extended')

    def split_list(iterable, n=1):
        "Convierte un iterable en una lista de listas con n valores."

        len_iterable = len(iterable)
        for ndx in range(0, len_iterable, n):
            yield iterable[ndx:min(ndx + n, len_iterable)]

    def read_metadata():
        "Descarga los metadatos de google sheets, filtra sólo aquellos con una fuente y crea una columna de status_id"

        meta = pd.read_csv(metadatos_url, parse_dates=['fecha'])
        meta = meta[(meta.fuente.notna()) & (meta.fuente.str.contains('twitter.com/'))]
        meta['status_id'] = meta['fuente'].apply(lambda x: x.split('/')[-1]).astype(int)
        return meta

    def get_statuses(fuentes):
        "Devuelve un diccionario de status de twitter donde la llave es el status_id"

        tw = auth_twitter()
        statuses = []
        for fuentes_seccion in split_list(fuentes, 50):
            statuses.extend(tw.GetStatuses(fuentes_seccion))
        return {status._json['id']:status._json for status in statuses}

    def archivar(meta, statuses):
        "Descarga la imagen de cada status de twitter, extrae los metadatos más significativos y sube todo al internet archive"

        for reporte in meta.to_dict(orient='records'):

            ia_identifier = 'vacunacioncovid19bolivia_{}'.format(reporte['fecha'].strftime('%Y%m%d'))
            print('https://archive.org/details/{} : {}'.format(ia_identifier, reporte['status_id']))
            
            status = statuses[reporte['status_id']]
            image_url = status['entities']['media'][0]['media_url']
            filename = '{}/{}.{}'.format(directory, ia_identifier, image_url.split('.')[-1])

            with open(filename, 'wb') as f:
                f.write(requests.get(image_url).content)

            ia_meta = {'title': 'Reporte de Vacunación de Covid-19 en Bolivia para el {}'.format(reporte['fecha'].strftime('%-d de %B, %Y')),
                       'description': status['full_text'],
                       'source': reporte['fuente'],
                       'creator': status['user']['name'],
                       'mediatype': 'image',
                       'collection': 'vacunacion-covid19-bolivia',
                       'date': reporte['fecha'].strftime('%Y-%m-%d')}

            ia.upload(ia_identifier,
                      filename,
                      metadata=ia_meta,
                      access_key = credenciales["ia"]["access"],
                      secret_key = credenciales["ia"]["secret"],
                      retries=4
                     )
    
    def filtrar_nuevos(meta):
        "Devuelve sólo las entradas que no hayan sido descargadas previamente."
        
        viejos = [dt.datetime.strptime(filename.split('.')[0].split('_')[1], '%Y%m%d') for filename in os.listdir(directory)]
        return meta[~meta.fecha.isin(viejos)]
    
    credenciales = load_credenciales()
    
    meta = read_metadata()
    if actualizar == True:
        meta = filtrar_nuevos(meta)
    if len(meta) > 0:
        statuses = get_statuses(meta['status_id'].tolist())
        archivar(meta, statuses)

def consolidar():
    """
    Sincroniza sheets
    """
    
    def download_sheet(url, filename):
        datos = requests.get(url).text
        datos = datos.replace('\r','')
        with open(filename, 'w+') as f:
            f.write(datos)
            
    for url, filename in zip([metadatos_url, datos_url], ['metadata.csv', 'vaccinations.csv']):
        download_sheet(url, filename)
        
def update():
    """
    Actualiza el repositorio
    """
    
    last_update = pd.read_csv('metadata.csv', parse_dates=['fecha']).fecha.max()
    repository = Repo('.')
    repository.git.add(all=True)
    repository.index.commit(last_update.strftime('%Y-%m-%d'))
    repository.remotes.origin.push()

In [4]:
archivar_reportes()
consolidar()

https://archive.org/details/vacunacioncovid19bolivia_20220412 : 1514051170737827851
https://archive.org/details/vacunacioncovid19bolivia_20220413 : 1514427495101067267
https://archive.org/details/vacunacioncovid19bolivia_20220414 : 1514788872248463370
https://archive.org/details/vacunacioncovid19bolivia_20220415 : 1515153239708160002
https://archive.org/details/vacunacioncovid19bolivia_20220416 : 1515514878357905408
https://archive.org/details/vacunacioncovid19bolivia_20220417 : 1515884171134214145
https://archive.org/details/vacunacioncovid19bolivia_20220419 : 1516603986673082372
https://archive.org/details/vacunacioncovid19bolivia_20220420 : 1516968665538727936
https://archive.org/details/vacunacioncovid19bolivia_20220421 : 1517318953147801601
https://archive.org/details/vacunacioncovid19bolivia_20220422 : 1517695232418226181
https://archive.org/details/vacunacioncovid19bolivia_20220423 : 1518031936291315713
https://archive.org/details/vacunacioncovid19bolivia_20220424 : 151841514222

In [5]:
update()